# Omegaconf 使用教程
比较灵活，支持支持YAML配置文件、对象、CLI参数等等

In [3]:
!pip install omegaconf

Looking in indexes: http://pypi.douban.com/simple
     |████████████████████████████████| 81kB 2.4MB/s eta 0:00:01
     |████████████████████████████████| 122kB 203kB/s eta 0:00:01
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-cp36-none-any.whl size=144574 sha256=c2c3b4607285f29e725394022abe4af191b9dd95245deb352dc51dba905caab0
  Stored in directory: /Users/yin/Library/Caches/pip/wheels/60/6d/f1/13417b608b58bd50da665b00f4a09422674bc4d800c09d06bd
Successfully built antlr4-python3-runtime


In [4]:
from omegaconf import OmegaConf

先来看一个例子

In [69]:
toy_dict = {"k" : "v", "list" : [1, {"a": "1", "b": "2", 3: "c"}]}
conf = OmegaConf.create(toy_dict)
print(OmegaConf.to_yaml(conf))

k: v
list:
- 1
- a: '1'
  b: '2'
  3: c



In [62]:
type(conf)

omegaconf.dictconfig.DictConfig

In [15]:
conf['k']

'v'

In [17]:
conf.k

'v'

In [19]:
conf.list[0]

1

## 如何构建一个 OmegaConf 类

### 构建一个空的

In [65]:
conf = OmegaConf.create()
print(OmegaConf.to_yaml(conf))

{}



### 将python字典转成 OmegaConf 类型

In [66]:
conf = OmegaConf.create({"k" : "v", "list" : [1, {"a": "1", "b": "2", 3: "c"}]})
print(OmegaConf.to_yaml(conf))

k: v
list:
- 1
- a: '1'
  b: '2'
  3: c



### 将python列表转成 OmegaConf 类型

In [67]:
conf = OmegaConf.create([1, {"a":10, "b": {"a":10, 123: "int_key"}}])
print(OmegaConf.to_yaml(conf))

- 1
- a: 10
  b:
    a: 10
    123: int_key



In [68]:
type(conf)

omegaconf.listconfig.ListConfig

### 从一个 yaml 文件来解析

In [22]:
conf = OmegaConf.load('example.yaml')
print(OmegaConf.to_yaml(conf))

server:
  port: 80
log:
  file: ???
  rotation: 3600
users:
- user1
- user2



### 从命令行参数来解析

In [23]:
%run example.py server.port=82 log.file=log2.txt

server:
  port: 82
log:
  file: log2.txt



### 从一个dataclass类

In [24]:
from dataclasses import dataclass
@dataclass
class MyConfig:
    port: int = 80
    host: str = "localhost"
conf = OmegaConf.structured(MyConfig(port=443))
print(OmegaConf.to_yaml(conf))

port: 443
host: localhost



## omegaconf 的一些基础操作

In [42]:
conf = OmegaConf.load('example.yaml')
print(OmegaConf.to_yaml(conf))

server:
  port: 80
log:
  file: ???
  rotation: 3600
users:
- user1
- user2



In [27]:
# ??? 的作用
conf.log.file

MissingMandatoryValue: Missing mandatory value: log.file
    full_key: log.file
    object_type=dict

In [28]:
conf.log.file = 'log.txt'
conf.log.file

'log.txt'

In [29]:
print(OmegaConf.to_yaml(conf))

server:
  port: 81
  hostname: localhost
log:
  file: log.txt
  rotation: 3600
users:
- user1
- user2
database:
  hostname: database01
  port: 3306



In [26]:
# 修改原来的值
conf.server.port = 81

# 增加新的键值对
conf.server.hostname = "localhost"
conf.database = {'hostname': 'database01', 'port': 3306}
print(OmegaConf.to_yaml(conf))

server:
  port: 81
  hostname: localhost
log:
  file: ???
  rotation: 3600
users:
- user1
- user2
database:
  hostname: database01
  port: 3306



In [43]:
conf.a.b.c = 12

ConfigAttributeError: Missing key a
    full_key: a
    object_type=dict

In [44]:
OmegaConf.update(conf, "a.b.c", 12, force_add=True)

In [45]:
conf

{'server': {'port': 80}, 'log': {'file': '???', 'rotation': 3600}, 'users': ['user1', 'user2'], 'a': {'b': {'c': 12}}}

In [47]:
conf_2 = OmegaConf.create({"name" : "root", "password": "123"})

In [48]:
merged_conf = OmegaConf.merge(conf, conf_2)

In [49]:
print(OmegaConf.to_yaml(conf))

server:
  port: 80
log:
  file: ???
  rotation: 3600
users:
- user1
- user2
a:
  b:
    c: 12



### 存入 yaml 文件

In [30]:
with open('save_example.yaml', 'w') as fp:
    OmegaConf.save(config=conf, f=fp.name)

## 值的引用，通过 $ 来实现

In [51]:
import yaml
import pprint
pp = pprint.PrettyPrinter(indent=4)
with open("ref_example.yaml", "r") as stream:
    conf = yaml.safe_load(stream)

In [38]:
conf['client']['server_port']

'${server.port}'

In [54]:
conf = OmegaConf.load('ref_example.yaml')
conf.client.server_port

80

In [52]:
print(OmegaConf.to_yaml(conf))

server:
  host: localhost
  port: 80
client:
  url: http://${server.host}:${server.port}/
  server_port: ${server.port}
  description: Client of ${.url}



In [60]:
OmegaConf.resolve(conf)

In [61]:
print(OmegaConf.to_yaml(conf))

server:
  host: localhost
  port: 80
client:
  url: http://localhost:80/
  server_port: 80
  description: Client of http://localhost:80/

